In [ ]:
from tope import Tope
from tope.net import *
from tope.orth import *

import json, os, numpy as np
with open("polys.json") as fd: polys = json.load(fd)

In [3]:
p_24cell = [
    [1, 0, 0, 0], [-1, 0, 0, 0],
    [0, 1, 0, 0], [0, -1, 0, 0],
    [0, 0, 1, 0], [0, 0, -1, 0], 
    [0, 0, 0, 1], [0, 0, 0, -1],
    [1, -1, 0, 0], [-1, 1, 0, 0], 
    [1, 0, -1, 0], [-1, 0, 1, 0], 
    [1, 0, 0, -1], [-1, 0, 0, 1], 
    [0, 1, -1, 0], [0, -1, 1, 0], 
    [0, 1, 0, -1], [0, -1, 0, 1],
    [1, 0, -1, -1], [-1, 0, 1, 1], 
    [0, 1, -1, -1], [0, -1, 1, 1],
    [1, 1, -1, -1], [-1, -1, 1, 1]
]

In [ ]:
# need to get the net in its own span
def put_in_own_span(N: Net):
    root_facet = N.tope.faces[-1][N.tree.root]
    ref_pt = N.tope.vertices[root_facet].mean(axis=0)
    
    offsets = [0] + [len(vertices) for vertices in N.facets.values()]
    offsets = np.cumsum(offsets)
    all_vertices = np.concatenate(list(N.facets.values()))
    all_vertices, basis = in_own_span(all_vertices - ref_pt)
                       
    # Need to reflect in one axis if orientation of root face is wrong.
    inward_normal = N.tope.vertices.mean(axis=0) - ref_pt
    if np.det(np.c_[basis.T, inward_normal]) < 1:
        all_vertices[:,d0] = -all_vertices[:,0]
    
    for i in N.facets:
        N.facets[i] = all_vertices[offsets[i]:offsets[i+1]]

In [ ]:
EdgeList = list[np.ndarray] # list of 2xdim arrays

def get_edges(N: Net) -> EdgeList: # apply to unfolded Net
    edges = []
    for i, vertices in N.facets.items():
        facet_template = N.tope.get_face(i) # has correct indices
        edges.extend((vertices[list(e)] for e in facet_template.faces[1]))
    return edges

FacetLabels = list[tuple[str, np.ndarray]]

def get_facet_labels(N: Net) -> FacetLabels:
    labels = []
    for i, vertices in N.facets.items():
        labels.append((N.tope.labels[-1][i], vertices.mean(axis=0)))
    return labels

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection # container for line segments
from matplotlib.text import Text, Annotation
import matplotlib.cm as cm # colour mappings
import random

edges = get_edges(N)
        
fig, ax = plt.subplots()

lc = LineCollection(edges)
ax.add_collection(lc)
for label, pos in get_facet_labels(N):
    ax.add_artist(Text(*pos, text=str(label)))

ax.autoscale()
fig.savefig("fig.png")
fig.show()

In [ ]:
def get_net_for_facet(P: Tope, i: int) -> Net:
    F=P.get_face(i)
    labels = F.labels
    F = F.in_own_span
    if F.vertices.shape[1] >= P.vertices.shape[1]:
        print("Failed: facet doesn't live in a subspace.")
        return False
    F.labels = labels
    
    G = get_facet_graph(F)
    T = G.get_spanning_tree(root=0, skip=set())
    N = Net(F, T)
    N.unfold()
    
    put_in_own_span(N)
    #print(N.facets)
    if N.facets[0].shape[1] > 2:
        print("Failed: net doesn't live in a plane.")
        return False
    
    return N

In [ ]:
MARGIN_FACTOR = 1.1


def save_nets(P: Tope, name: str, margin_factor: float = MARGIN_FACTOR, dpi: int = 300):
    if os.path.exists(name):
        logger.error("File already exists. Aborting.")
        return
    os.mkdir(name)
    
    nfacets = len(P.faces[-1])
    facet_net_list = []
    
    xlim = [0,0]
    ylim = [0,0]

    for i in range(nfacets):
        N = get_net_for_facet(P, i)
        for face_v in N.facets.values():
            lower, upper = face_v.min(axis=0), face_v.max(axis=0)
            xlim[0] = min(xlim[0], lower[0])
            ylim[0] = min(ylim[0], lower[1])
            xlim[1] = max(xlim[1], upper[0])
            ylim[1] = max(ylim[1], upper[1])
        facet_net_list.append(N)
        
    xlim[0] *= margin_factor
    ylim[0] *= margin_factor
    xlim[1] *= margin_factor
    ylim[1] *= margin_factor
        
    for i in range(nfacets):
        fig, ax = plt.subplots()
        N = facet_net_list[i]
        lc = LineCollection(get_edges(N), colors="grey")
        ax.add_collection(lc)
        for label, pos in get_facet_labels(N):
            ax.add_artist(Text(pos[0]-0.05, pos[1], text=str(label), fontsize=6, ha="center", va="center"))

        ax.set_xlim(*xlim)
        ax.set_ylim(*ylim)
        
        fig.savefig(os.path.join(name, f"{i}.png"), dpi=dpi)

In [ ]:
save_nets(Tope.from_vertices(polys["2070"]), "figs/2070")

In [ ]:
for name, poly in polys.items():
    save_nets(Tope.from_vertices(poly), f"figs/{name}")

In [ ]:
ID = ""
with open("../polys1.json") as fd: data = json.load(fd)
save_nets(Tope.from_vertices(data[ID]), f"figs/{ID}")